# Ising-Lenz Ergodicity: Analysis magnetisation autocorrelation

    
     (c) 2013, 2014, 2015, 2016, 2025 Süzen
     GPL v3 

We computed average magnetisation autocorrelations across sizes and temperatures at $H=1.0$ and $J=0$

Using autocorrelation curves, we identify autocorrelation times via differencing algorithm. The algorithm finds the lagged difference of the autocorrelation and report the relaxation time at a given epsilon threshold. `epsilon=1e-8, detect_lag=10`, also report bias corrected bootstrapped 95% confidence intervals. 

We plotted all autocorrelation and relaxation times under `plots/magnetisation` at all sizes and temperatures. Combined plots goes into `plots/magnetisationOverTemp`. 

## Analysis methods and loads

We provide analysis methods for easy of generation.

In [ ]:
rm(list=ls()) 
require("isingLenzMC"); # Developed and tested on v0.2.5 and R v4.5.1
library(parallel)       # Core package

source("../src/power_utilities.R");

In [ ]:
autocorrelation_time_ci <- function(result, epsilon=1e-8, detect_lag=10) {
    # Self time correaltion on result, return log(t), base e!
    autocorr_times <- vector("numeric", result$nrepeat)
    for(i in 2:result$nrepeat) { # skip first trajectory
        observe_mag <- result$average_magnetisation_repeats[[i]]
        c_t <- self_time_correlation(observe_mag)
        t <- result$mc_time
        y <- log10(c_t)
        x <- log10(t)
        ix <- which(abs(diff(y, lag=detect_lag)) < epsilon)[1]
        ixl <- as.numeric(t[ix])
        autocorr_times[i] <- log10(ixl)
    }
    ix <- which(!is.na(autocorr_times))
    get_mean_boot95_bca(autocorr_times[ix])
}

get_min_max_autocorr <- function(result) {
    # Self time-correlation on result
    # on log(c_t)
    autocorr_times <- vector("numeric", result$nrepeat)
    observe_mag <- result$average_magnetisation_repeats[[2]]
    c_t <- self_time_correlation(observe_mag)
    y <- log(c_t)
    minct <- min(y)
    maxct <- max(y)
    for(i in 3:result$nrepeat) {
        observe_mag <- result$average_magnetisation_repeats[[i]]
        c_t <- self_time_correlation(observe_mag)
        y <- log(c_t)
        minct_ <- min(y)
        maxct_ <- max(y)
        if(minct_ <= minct) minct <- minct_
        if(maxct_ >= maxct) maxct <- maxct_
    }
    c(minct, maxct)
}

info_autocorrelation_time <- function(result) {
  # Get Autocorrelation time simpy testing for epsilon on difference.
  # Compute CI via bCA
    ac_time <- autocorrelation_time_ci(result)
    if(result$transP == 1) dynamics <- "Metropolis"
    if(result$transP == 2) dynamics <- "Glauber"
    as.data.frame(list("N"=result$N, "beta"=result$ikBT, "J"=result$J, "H"=result$H, 
        "dynamics"=dynamics, "transP"=result$transP,
        "ac_time_mean"=ac_time$mean, "ac_time_upper"=ac_time$upper, 
        "ac_time_lower"=ac_time$lower))
}

plot_ac_time <- function(result, base_plot="../plots/magnetisation/") {
  # Plot autocorrelation times
  if(result$transP == 1) dynamics <- "Metropolis"
  if(result$transP == 2) dynamics <- "Glauber"
  fname <- paste(base_plot, dynamics, "N", result$N, "beta", result$ikBT, ".pdf", sep = "")
  pdf(fname, pointsize = 18) # pointsize will effect all label and title font sizes.
  minmaxmag <- get_min_max_autocorr(result)
  bci <- autocorrelation_time_ci(result)
  for (i in 2:result$nrepeat) {
    observe_mag <- result$average_magnetisation_repeats[[i]]
    c_t <- self_time_correlation(observe_mag)
    t <- result$mc_time
    y <- log10(c_t)
    x <- log10(t)
    if (i == 2) {
      plot(x, y, lty = 1, lwd = 0.02, type = "l", ylim = minmaxmag, ann=FALSE,
          cex.lab = 1.2,  cex.axis = 1.2, cex.main = 1, cex.sub = 1)
    }
    if (i > 2) {
      lines(x, y, lty = 1, lwd = 0.02, type = "l", ylim = minmaxmag)
    }
  }
  abline(v = bci$mean, lty=2, lwd = 3)
  abline(v = bci$upper, lty=3, lwd = 3)
  abline(v = bci$lower, lty=3, lwd = 3)
  xlabel <- "Log Scale MC Time Steps"
  ylabel <- "Log Scale Autocorrelation"
  title_main <- bquote(paste("Average Magnetisation Autocorrelation"))
  N    <- result$N
  ikBT <- result$ikBT
  subtitle <- bquote(paste(
    .(dynamics), " N=", .(N), " ", beta, "=", .(ikBT)
  ))
  title(title_main, xlab=xlabel, ylab=ylabel)
  mtext(subtitle)
    legend("bottomright",
      legend = c("Time Correlation Curves", "Autorcorrelation Time (Mean)", "Autorcorrelation Time (CI)"),
      lty = c(1, 2, 3),
      lwd = 3,
      bty = "n",
    )
  dev.off()

}

get_ac_times_df <- function(average_magnetisation_data, iplot=TRUE, base_plot = "../plots/magnetisation/") {
  # Compute autocorrelation and return info data.frame
  # Plot if asked.
    result <- average_magnetisation_data[[1]]
    df_ac_time <- info_autocorrelation_time(result)
    for(i in 2:length(average_magnetisation_data)) {
        result <- average_magnetisation_data[[i]]
        df_ac_time <- rbind.data.frame(df_ac_time, info_autocorrelation_time(result))
        if(iplot) plot_ac_time(result, base_plot)
    }
    df_ac_time
}

get_magnetisation_ac_over_temp <- function(df_ac_time, N, transP) {
    df <- df_ac[df_ac['N'] == N & df_ac['transP'] == transP,] [,c('beta', 'ac_time_mean','ac_time_lower', 'ac_time_upper')]
    x <- df['beta'][,1]
    y <- df['ac_time_mean'][,1]
    y_lower <- df['ac_time_lower'][,1]
    y_upper <- df['ac_time_upper'][,1]
    list("x"=x, "y"=y, "y_lower"=y_lower,"y_upper"=y_upper)
}

magnetisation_ac_over_temp <- function(df_ac_time, N, transP, pcount=1, base_plot="../plots/magnetisationOverTemp/") {
    if(transP == 1) dynamics <- "Metropolis"
    if(transP == 2) dynamics <- "Glauber"
    fname <- paste(base_plot, dynamics, N, ".pdf", sep = "")
    pdf(fname, pointsize = 18) # pointsize will effect all label and title font sizes.
    val <- get_magnetisation_ac_over_temp(df_ac_time, N, transP)
    plot(val$x, val$y, lty=1, type="p", pch=pcount, ann=FALSE, ylim=c(3.0, 6.1))
    arrows(val$x, val$y_lower, val$x, val$y_upper, length=0.05, angle=90, code=3)
    xlabel <- "Temperature Range"
    ylabel <- "Log Scale Autocorrelation MC Time"
    title_main <- bquote(paste("Autocorrelation Times"))
    subtitle <- bquote(paste(
        "Average Magnetisation: ", .(dynamics), " N=", .(N)
    ))
    title(title_main, xlab=xlabel, ylab=ylabel)
    mtext(subtitle)
    dev.off()
    list("x"=val$x, "y"=val$y, "y_lower"=val$y_lower,"y_upper"=val$y_upper)
}


## Computation

We compute and pooled the autocorrelation times for each case using the generated dataset.

In [ ]:
# Load dataset
# Compute autocorrelation times
average_magnetisation_data <- readRDS("../datasets/ising1Dmagnetisation.rds")
df_ac <- get_ac_times_df(average_magnetisation_data, iplot = TRUE)

## Plots

Mentioned plots as in introduction.

In [ ]:
N <- 512
transP <- 1
val512_1 <- magnetisation_ac_over_temp(df_ac_time, N, transP)
transP <- 2
val512_2 <- magnetisation_ac_over_temp(df_ac_time, N, transP, pcount = 2)
N <- 1024
transP <- 1
val1024_1 <- magnetisation_ac_over_temp(df_ac_time, N, transP, pcount = 3)
transP <- 2
val1024_2 <- magnetisation_ac_over_temp(df_ac_time, N, transP, pcount = 4)
N <- 1536
transP <- 1
val1536_1 <- magnetisation_ac_over_temp(df_ac_time, N, transP, pcount=5)
transP <- 2
val1536_2 <- magnetisation_ac_over_temp(df_ac_time, N, transP, pcount=6)


In [ ]:
# Metropolis
base_plot <- "../plots/magnetisationOverTemp/"
fname <- paste(base_plot, "ac_times_metropolis.pdf", sep = "")
pdf(fname, pointsize = 18) # pointsize will effect all label and title font sizes.
val <- val512_1
plot(val$x, val$y, lty=1, pch=1, type="p", col="black", ann=FALSE, ylim=c(3.0, 6.1), xlim=c(0.45, 1.55))
arrows(val$x, val$y_lower, val$x, val$y_upper, col="black",length=0.05, angle=90, code=3)
val <- val1024_1
lines(val$x, val$y, lty=2, pch=2, type="p", col="blue",ann=FALSE, ylim=c(3.0, 6.1), xlim=c(0.45, 1.55))
arrows(val$x, val$y_lower, val$x, val$y_upper, col="blue", length=0.05, angle=90, code=3)
val <- val1536_1
lines(val$x, val$y, lty=3, pch=3, type="p", col="red", ann=FALSE, ylim=c(3.0, 6.1), xlim=c(0.45, 1.55))
arrows(val$x, val$y_lower, val$x, val$y_upper, col="red", length=0.05, angle=90, code=3)
xlabel <- "Temperature Range"
ylabel <- "Log Scale Autocorrelation MC Time"
title_main <- bquote(paste("Autocorrelation Times"))
subtitle <- bquote(paste(
    "Average Magnetisation: Metropolis"
))
title(title_main, xlab=xlabel, ylab=ylabel)
mtext(subtitle)
  legend("bottomleft",
    legend = c("N=512", "N=1024", "N=1536"),
    pch = c(1, 2, 3),
    col = c("black", "blue", "red"),
    lwd = 2,
    bty = "n",
  )
dev.off()

In [ ]:
# Glauber
base_plot <- "../plots/magnetisationOverTemp/"
fname <- paste(base_plot, "ac_times_glauber.pdf", sep = "")
pdf(fname, pointsize = 18) # pointsize will effect all label and title font sizes.
val <- val512_2
plot(val$x, val$y, lty=1, pch=1, type="p", col="black", ann=FALSE, ylim=c(3.0, 6.1), xlim=c(0.45, 1.55))
arrows(val$x, val$y_lower, val$x, val$y_upper, col="black",length=0.05, angle=90, code=3)
val <- val1024_2
lines(val$x, val$y, lty=2, pch=2, type="p", col="blue",ann=FALSE, ylim=c(3.0, 6.1), xlim=c(0.45, 1.55))
arrows(val$x, val$y_lower, val$x, val$y_upper, col="blue", length=0.05, angle=90, code=3)
val <- val1536_2
lines(val$x, val$y, lty=3, pch=3, type="p", col="red", ann=FALSE, ylim=c(3.0, 6.1), xlim=c(0.45, 1.55))
arrows(val$x, val$y_lower, val$x, val$y_upper, col="red", length=0.05, angle=90, code=3)
xlabel <- "Temperature Range"
ylabel <- "Log Scale Autocorrelation MC Time"
title_main <- bquote(paste("Autocorrelation Times"))
subtitle <- bquote(paste(
    "Average Magnetisation: Glauber"
))
title(title_main, xlab=xlabel, ylab=ylabel)
mtext(subtitle)
  legend("bottomleft",
    legend = c("N=512", "N=1024", "N=1536"),
    pch = c(1, 2, 3),
    col = c("black", "blue", "red"),
    lwd = 2,
    bty = "n",
  )
dev.off()